<a href="https://colab.research.google.com/github/KendallScott/Capstone_alt/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [3]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

In [4]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [5]:
pip install eemont

In [6]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [8]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7AYATCQ_MpfpEY_DId3H2eYhzEzRxQaj7qNvpz6jY7I&tc=6ZKiBQMvq_7gEXSqQQ9lfcL7IBWo4WWolnW_WNFkeZY&cc=IrtUSusXFMBTfJHVpF4YywCR5UuEdzODKd29-Y817p8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhWgUzwrhPgMWvGPtIG4fXt0AyvM7V5gJxujUiDI9IDeSHsaZYTa7GA

Successfully saved authorization token.


In [82]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')


# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))
mines.sample(3)

Data dimensions: (25124, 31)


,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
1435,1436,TEECO COAL CO,TEECO COAL CO,7,AREA 1,78-162,NaN,NaN,TN78-162,NaN,...,NaN,NaN,NaN,NaN,NaN,4,https://www.osmre.gov/,8305.992241,521572.376424,"MULTIPOLYGON (((-9476475.663 4244668.950, -947..."
15053,15054,P A CONSTR CO,P A CONSTR CO,5,NaN,8670047,NaN,NaN,KY8670047,ELKHORN #3,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,2150.003325,20131.483033,"MULTIPOLYGON (((-9194502.391 4468159.081, -919..."
9956,9957,ANDERSON COAL CO INC,ANDERSON COAL CO INC,5,NaN,4605067,8567,NaN,KY4605067,HAZARD #4,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,2942.049059,27648.594140,"MULTIPOLYGON (((-9224727.212 4488641.998, -922..."


In [84]:
mines.sample(10)

,OBJECTID,permittee,company,coalmine_op_status,mine_name,permit_id,msha_id,state_tribe_mine_id,national_id,coal_bed_names,...,mountaintop,steep_slope,highwall,auger,comment,contact,information_link,shape_Length,shape_Area,geometry
12917,12918,SAVOY COALS INC,SAVOY COALS INC,4,NaN,687877X,NaN,NaN,KY687877X,"BLUE GEM SEAM, JELLICO SEAM",...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,7671.550582,4.132450e+05,"MULTIPOLYGON (((-9389380.611 4387498.056, -938..."
15374,15375,REEDY COAL CO INC,REEDY COAL CO INC,7,NaN,8675045,NaN,NaN,KY8675045,W'BURG,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,6390.300282,6.453841e+05,"MULTIPOLYGON (((-9219675.708 4465094.759, -921..."
6308,6309,NaN,,4,NaN,0630186,NaN,NaN,KY0630186,,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,1736.850865,1.316564e+05,"MULTIPOLYGON (((-9366335.032 4457994.908, -936..."
23242,23243,"ERP ENVIRONMENTAL FUND, INC.",NaN,7,EUROPA MINE,U502499,NaN,NaN,WVU502499,NaN,...,0.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,155351.292452,1.178205e+06,"MULTIPOLYGON (((-9086291.696 4563679.691, -908..."
13714,13715,BUCK COAL INC,BUCK COAL INC,7,NaN,8360156,15784,NaN,KY8360156,ELK #3,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,14037.121664,8.989250e+05,"MULTIPOLYGON (((-9204485.393 4505279.807, -920..."
20435,20436,NaN,,2,NaN,8984495,NaN,NaN,KY8984495,,...,0.0,0.0,NaN,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,308.430636,5.945543e+03,"MULTIPOLYGON (((-9163159.704 4535679.980, -916..."
21556,21557,"HUTCHINSON MINERALS, LLC","A&E CONST., INC.",7,5 BLOCK SURFACE MINE #1,S303392,NaN,NaN,WVS303392,NaN,...,1.0,0.0,NaN,0.0,NaN,2,https://dep.wv.gov/dmr/Pages/default.aspx,3799.018752,9.649264e+04,"MULTIPOLYGON (((-9009855.520 4626859.228, -900..."
11603,11604,JACK LITTLE,JACK LITTLE,7,NaN,8360099,NaN,NaN,KY8360099,#2 ELKHORN,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,929.214359,1.935705e+04,"MULTIPOLYGON (((-9201661.039 4513819.143, -920..."
25048,25049,Mid-South Mining Inc,Mid-South Mining Inc,7,Stevenson Mine,P3515,NaN,P3515,ALP3515,NaN,...,NaN,NaN,NaN,NaN,State defined mine status: Closed. May be eith...,8,https://surface-mining.alabama.gov/,503.866583,1.544037e+04,"MULTIPOLYGON (((-9556657.075 4156653.502, -955..."
8071,8072,RICHLAND COAL COMPANY,RICHLAND COAL COMPANY,6,NaN,1180123,NaN,NaN,KY1180123,DEAN 1880,...,0.0,0.0,0.0,0.0,NaN,3,https://eec.ky.gov/Natural-Resources/Mining/Pa...,8466.537181,3.087484e+05,"MULTIPOLYGON (((-9374284.981 4396440.384, -937..."


In [85]:

mines = mines.drop(columns=['permittee'])
mines = mines.drop(columns=['mine_name'])
mines = mines.drop(columns=['msha_id'])
mines = mines.drop(columns=['state_tribe_mine_id'])
mines = mines.drop(columns=['mountaintop'])
mines = mines.drop(columns=['steep_slope'])
mines = mines.drop(columns=['highwall'])
mines = mines.drop(columns=['auger'])
mines = mines.drop(columns=['comment'])
mines = mines.rename(columns={'OBJECTID': 'shapeName'})

In [90]:
mines = mines.drop(columns=['coal_bed_names'])
mines = mines.drop(columns=['date_of_map'])
mines = mines.drop(columns=['inspectable_unit_status'])
mines = mines.drop(columns=['permit_approval_date'])
mines = mines.drop(columns=['edit_date'])
mines = mines.drop(columns=['area_mine'])
mines = mines.drop(columns=['contour'])

In [92]:
mines = mines.drop(columns=['permit_application_type'])
mines = mines.drop(columns=['reported_area'])

In [94]:
mines = mines.drop(columns=['permit_application_date'])

In [95]:
mines

,shapeName,company,coalmine_op_status,permit_id,national_id,post_smcra,calculated_area,contact,information_link,shape_Length,shape_Area,geometry
0,1,HIGGINS COAL CO INC,7,2404,TN2404,1.0,2.220938,4,https://www.osmre.gov/,1220.769913,1.354382e+04,"MULTIPOLYGON (((-9515951.177 4212750.257, -951..."
1,2,MIDNIGHT COAL CO,7,2405,TN2405,1.0,2.752046,4,https://www.osmre.gov/,860.404674,1.678249e+04,"MULTIPOLYGON (((-9516024.476 4212708.540, -951..."
2,3,META ELKHORN MINING CO INC,7,2756,TN2756,1.0,270.299198,4,https://www.osmre.gov/,7219.650616,1.689153e+06,"MULTIPOLYGON (((-9459511.615 4347543.957, -945..."
3,4,JORDAN BROTHERS COAL CO INC,7,2497,TN2497,1.0,3.507990,4,https://www.osmre.gov/,688.042643,2.188255e+04,"MULTIPOLYGON (((-9372128.011 4337061.595, -937..."
4,5,DOUBLE Q INC,7,2284023,TN2284023,1.0,13.751147,4,https://www.osmre.gov/,7994.318732,8.636343e+04,"MULTIPOLYGON (((-9344712.695 4372890.512, -934..."
...,...,...,...,...,...,...,...,...,...,...,...,...
25119,25120,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,154.872159,8,https://surface-mining.alabama.gov/,5370.984081,9.330482e+05,"MULTIPOLYGON (((-9540804.196 4143151.210, -954..."
25120,25121,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,2.410359,8,https://surface-mining.alabama.gov/,534.608228,1.452293e+04,"MULTIPOLYGON (((-9541424.447 4142622.814, -954..."
25121,25122,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,2.430763,8,https://surface-mining.alabama.gov/,507.253927,1.464629e+04,"MULTIPOLYGON (((-9540236.664 4142842.258, -954..."
25122,25123,Justice Coal Of Alabama LLC,1,P3946,ALP3946,1.0,116.325330,8,https://surface-mining.alabama.gov/,4223.617167,7.008642e+05,"MULTIPOLYGON (((-9540725.233 4143212.597, -954..."


In [96]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [ ]:
Map = geemap.Map()
Map.centerObject(region)

In [ ]:
# Create Dynamic World land cover composite
# Set the date range
start_date = '2021-01-01'
end_date = '2022-01-01'

landcover = geemap.dynamic_world(region, start_date, end_date, return_type='hillshade')
Map.addLayer(landcover, {}, 'Land Cover')

In [ ]:
# Visualize map
Map

In [ ]:
df = pd.DataFrame()

In [ ]:
mines_exploded=mines.explode()
mines_exploded

In [ ]:
i=0

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
 try:
        # Get dynamic world image collection by asset ID
        dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)

        # get band names, equal to column names
        bandnames = dc.first().bandNames().getInfo()[:-1]

        polygon = mines_exploded['geometry'].values[i]

        # Extract the coordinates from the polygon
        coordinates = polygon.exterior.coords[:]

        # Create an ee.Geometry object from the coordinates
        polygon = ee.Geometry.MultiPolygon(coordinates)

        # Set the maxError parameter
        max_error = 1  # Specify your desired maxError value

        # Project the geometry coordinates
        projected_polygon = polygon.transform('EPSG:4326', max_error)

        #decrease scale to get more rows of data/granularity
        ac = dc.getRegion(projected_polygon, scale=100 ).getInfo()
        df_constructor =pd.DataFrame(ac)

        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['company'] = mines_exploded['company'].values[i]
        df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
        df_constructor['start_date'] = i_date
        df_constructor['end_Date'] = f_date
        df = df.append(df_constructor)

        print(mines_exploded['company'].values[i])
        i+=1
 except:
          i+=1
          pass



In [78]:
df['company'].unique()

array([nan, 'COALCO INC', 'ENCOL INC', 'DEBCO COAL CO',
       'B + J EXCAVATING CO INC (TN)', 'CUMBERLAND COAL CO LLC',
       'PREMIUM COAL CO. INC', 'ROBINSON COAL CO',
       'SEQUATCHIE VALLEY COAL CORP', 'WHITE OAK COAL CORP',
       'COAL ENERGY INC', 'D + H COAL CO INC',
       'META ELKHORN MINING CO INC', 'ESTATE OF BRUNO GERNT INC',
       'KOPPER GLO FUEL INC', 'COLQUEST ENERGY INC',
       'EAST TENNESSEE RESOURCES INC', 'PLYMOUTH COAL CO',
       'S + H MINING INC', 'HAMBLIN COAL CO',
       'OLIVER SPRINGS MINING CO INC', 'TENNESSEE CONSOLIDATED COAL C0',
       'MARION COAL CO INC', 'PATTON MINING CO',
       'ADDINGTON ENTERPRISES INC', 'LCC TENNESSEE  LLC',
       'REX MINING INC', 'TRIPLE H COAL  LLC.', 'TENNESSEE MINING INC',
       'STONEY RIDGE COAL COMPANY INC', 'S R S COAL INC',
       'NATIONAL COAL CORPORATION', 'U S COAL INC', 'SHEMCO INC',
       'VOLUNTEER PROCESSING INC', 'MEADOWS COAL CO',
       'INDUSTRIAL PROCESSING COMPANY', 'ROBERT CLEAR COAL CORP',


In [79]:
df

,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,label,company,shapeName
1,20210104T162701_20210104T162655_T16SGF,-84.313178,36.291039,1609778000459,0.04542,0.274786,0.092658,0.039822,0.038571,0.303898,0.032706,0.121162,0.034524,1,NaN,NaN
2,20210104T162701_20210104T162655_T17SKA,-84.313178,36.291039,1609777999068,0.042026,0.290325,0.087177,0.038763,0.04232,0.29151,0.03383,0.12493,0.034269,5,NaN,NaN
3,20210129T162529_20210129T163010_T16SGF,-84.313178,36.291039,1611937999778,0.046217,0.258137,0.105504,0.026195,0.042333,0.265838,0.038878,0.15122,0.041957,5,NaN,NaN
4,20210129T162529_20210129T163010_T17SKA,-84.313178,36.291039,1611937998386,0.042931,0.266436,0.109466,0.02528,0.046427,0.248172,0.035881,0.158534,0.039877,5,NaN,NaN
1,20210104T162701_20210104T162655_T16SGF,-84.539553,36.298226,1609778000459,0.050889,0.388929,0.057669,0.038968,0.028941,0.279445,0.027273,0.086808,0.031611,1,COALCO INC,0000000000000000000f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,20210104T162701_20210104T162655_T16SGF,-84.345068,36.2852,1609778000459,0.069676,0.498362,0.074818,0.040029,0.038362,0.145228,0.039846,0.053292,0.03785,1,U S COAL INC,00000000000000000003
30,20210104T162701_20210104T162655_T16SGF,-84.344169,36.2852,1609778000459,0.069764,0.354117,0.079024,0.039367,0.043649,0.254445,0.040096,0.069402,0.044334,1,U S COAL INC,00000000000000000003
31,20210104T162701_20210104T162655_T16SGF,-84.345966,36.286098,1609778000459,0.055175,0.222323,0.069861,0.037601,0.052108,0.207541,0.224025,0.066515,0.042887,6,U S COAL INC,00000000000000000003
32,20210104T162701_20210104T162655_T16SGF,-84.345068,36.286098,1609778000459,0.069354,0.358745,0.090561,0.045413,0.046053,0.223375,0.054247,0.065136,0.039227,1,U S COAL INC,00000000000000000003


In [81]:
len(df['company'].unique())

48

In [80]:
len(mines_exploded['company'].unique())

27